# Logic trees for Nath & Thingbaijam (2012)

Read logic tree tables extracted from publication, along with a table of supplementary information write them to TEX and XML. Note that the logic trees require information from the source models, so if changes are made there then this must be rerun too.

Note: For imports to work, ../utilities directory must be added to PYTHONPATH

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from IPython.display import display

import pandas as pd

from openquake.hazardlib import nrml

import logic_tree_tools as ltt

In [4]:
# flag determines whether branch IDs are included in PDF drawings of logic trees
include_ids=False

## GMPE Logic Tree

In [5]:
ltt.nrml_to_pdf('example_gmpe_logic_tree.xml', include_ids=include_ids)

FileNotFoundError: [Errno 2] No such file or directory: 'example_gmpe_logic_tree.xml'

In [ ]:
gsim_tree_tsv = 'gmpe_logic_tree.tsv'
gsim_tree_df = ltt.read_tree_tsv(gsim_tree_tsv)
gsim_tree_df.drop(['Tectonic Region Type', 'Qualifier', 'Short Names'], axis=1)

In [ ]:
gsim_tree_deprecated_xml = gsim_tree_tsv.replace('.tsv', '_deprecated.xml')
ltt.write_gsim_tree_nrml(gsim_tree_df, gsim_tree_deprecated_xml, validate=True)
gsim_tree_deprecated = nrml.read(gsim_tree_deprecated_xml)

In [ ]:
new_models = [
    'SharmaEtAl2009',
    'Kanno2006Shallow', 
    'Kanno2006Deep',
    'NathEtAl2012Lower', 
    'NathEtAl2012Upper',
    'RaghukanthIyengar2007',
    'Gupta2010SSlab',
    'AtkinsonBoore2003SSlabJapan',
    'AtkinsonBoore2003SSlabCascadia']
gsim_tree_omit_new = ltt.df_to_tree(gsim_tree_df, omit=new_models, validate=True)
gsim_tree_omit_new_xml = gsim_tree_tsv.replace('.tsv', '_omit_new.xml')
with open(gsim_tree_omit_new_xml, 'wb+') as file:
    nrml.write([gsim_tree_omit_new], output=file, fmt='%g')
ltt.nrml_to_pdf(gsim_tree_omit_new_xml, include_ids=include_ids)

In [ ]:
gsim_tree_read = nrml.read(gsim_tree_omit_new_xml)

In [ ]:
# it used to be that you didn't get the same thing reading and writing
print(gsim_tree_omit_new
      .logicTreeBranchingLevel
      .logicTreeBranchSet
      .logicTreeBranch
      .uncertaintyModel
      .text)
print(gsim_tree_read
      .logicTree
      .logicTreeBranchingLevel
      .logicTreeBranchSet
      .logicTreeBranch.uncertaintyModel
      .text.strip())
print(gsim_tree_deprecated
      .logicTree
      .logicTreeBranchingLevel
      .logicTreeBranchSet
      .logicTreeBranch
      .uncertaintyModel
      .text.strip())

In [ ]:
node = gsim_tree_omit_new[0][0]
print(node)
print('%s = %dx %s' % (ltt.strip_fqtag(node.tag), len(node),
                       ltt.strip_fqtag(node[0].tag)))
print(ltt.get_dict_key_match(node.attrib, 'id'))

In [ ]:
# let's also write an "full" version of the tree
gsim_tree = ltt.df_to_tree(gsim_tree_df, validate=False)
gsim_tree_xml = gsim_tree_tsv.replace('.tsv','.xml')
with open(gsim_tree_xml, 'wb+') as f:
    nrml.write([gsim_tree], f, fmt='%g')
ltt.nrml_to_pdf(gsim_tree_xml, include_ids=include_ids)

## Source Model Logic Tree

In [ ]:
ltt.nrml_to_pdf('example_source_model_logic_tree.xml', include_ids=include_ids)

In [ ]:
source_tree_tsv_list = [
    'areal_model_logic_tree.tsv', 
    'source_model_logic_tree.tsv', 
    'smoothed_model_logic_tree.tsv',
    'no_fmd_uncertainty.tsv',
    'collapsed.tsv',
]
for source_tree_tsv in source_tree_tsv_list:
    print("PROCESSING: " + source_tree_tsv)
    source_tree_symbolic_df = ltt.read_tree_tsv(source_tree_tsv)
    display(source_tree_symbolic_df)
    
    source_tree_symbolic = ltt.df_to_tree(source_tree_symbolic_df, validate=True)
    source_tree_symbolic_xml = source_tree_tsv.replace('.tsv','_symbolic.xml')
    with open(source_tree_symbolic_xml, 'wb+') as f:
        nrml.write([source_tree_symbolic], f, fmt='%g')
    ltt.nrml_to_pdf(source_tree_symbolic_xml, include_ids=include_ids)
        
    source_tree_df = ltt.expand_sources(source_tree_symbolic_df)
    display(pd.concat((source_tree_df.head(), source_tree_df.tail())))
    source_tree = ltt.df_to_tree(source_tree_df, validate=False)
    source_tree_xml = source_tree_tsv.replace('.tsv','.xml')
    with open(source_tree_xml, 'wb+') as f:
        nrml.write([source_tree], f, fmt='%g')
    ltt.nrml_to_pdf(source_tree_xml, include_ids=include_ids)
    print('')


In [ ]:
node = source_tree_symbolic[0][0]
print(node)
print('%s = %dx %s' % (ltt.strip_fqtag(node.tag), len(node),
                       ltt.strip_fqtag(node[0].tag)))
print(ltt.get_dict_key_match(node.attrib, 'id'))
print(node[0].uncertaintyModel)

In [ ]:
node = source_tree[0][0]
print(node)
print('%s = %dx %s' % (ltt.strip_fqtag(node.tag), len(node),
                       ltt.strip_fqtag(node[0].tag)))
print(ltt.get_dict_key_match(node.attrib, 'id'))
model = node[0].uncertaintyModel
print(model)